In [ ]:
import nannyml as nml
import pandas as pd
from IPython.display import display

reference_df = nml.load_synthetic_binary_classification_dataset()[0]
analysis_df = nml.load_synthetic_binary_classification_dataset()[1]
display(reference_df.head())

feature_column_names = [
    col for col in reference_df.columns if col not in [
    'timestamp', 'y_pred_proba', 'period', 'y_pred', 'work_home_actual', 'identifier'
]]
calc = nml.UnivariateStatisticalDriftCalculator(
    feature_column_names=feature_column_names,
    timestamp_column_name='timestamp'
)

calc.fit(reference_df)
results = calc.calculate(analysis_df)
display(results.data.iloc[:, :9])
display(calc.previous_reference_results.iloc[:, :9])

for feature in calc.feature_column_names:
    drift_fig = results.plot(
        kind='feature_drift',
        feature_column_name=feature,
        plot_reference=True
    )
    drift_fig.show()

for cont_feat in calc.continuous_column_names:
    figure = results.plot(
        kind='feature_distribution',
        feature_column_name=cont_feat,
        plot_reference=True
    )
    figure.show()

for cat_feat in calc.categorical_column_names:
    results.plot(
        kind='feature_distribution',
        feature_column_name=cat_feat,
        plot_reference=True).show()

ranker = nml.Ranker.by('alert_count')
ranked_features = ranker.rank(results, only_drifting = False)
display(ranked_features)